Set up:

- Cross validation: grid

- Period: 2017 and fwd

- Hyperparameter smoothing: none

- Fallback: cash

- Daily trading: Yes (shift_series=None)

In [20]:
import sys, os, pickle
from joblib import Parallel, delayed
import pandas as pd
from jumpmodels.utils import filter_date_range
from jumpmodels.preprocess import StandardScalerPD, DataClipperStd
from jumpmodels.sparse_jump import SparseJumpModel
from sklearn.preprocessing import StandardScaler
from pypfopt.black_litterman import BlackLittermanModel, market_implied_prior_returns
from pypfopt.efficient_frontier import EfficientFrontier
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import exceptions 
from scipy.optimize import brentq
import cvxpy as cp
from scipy import stats

sys.path.append('/Users/victor/Documents/thesis_vri_vp/vic_new')         # for mac
#sys.path.append('C:\\Users\\victo\\git_new\\thesis_vri_vp\\vic_new')      # for windows
from feature_set_v2 import MergedDataLoader 

In [11]:
# 0) Global parameters ------------------------------------------------------------------
REFIT_FREQ        = "ME"        
MIN_TRAINING_YEARS= 8
MAX_TRAINING_YEARS= 12
INITIAL_TRAIN_START = "2002-05-31"
test_start        = "2014-03-01"

# Pick method to drive backtest: "grid", "bayes" or "history" ---------------------------
cv_choice = "grid"

# Paths & tickers -----------------------------------------------------------------------
script_dir = os.getcwd()
base_dir   = os.path.abspath(os.path.join(script_dir, "..", "..", ".."))
data_dir   = os.path.join(base_dir, "data_new")

factor_file = os.path.join(data_dir, "1estimation_index_returns.csv")
market_file = os.path.join(data_dir, "1macro_data.csv")
etf_file    = os.path.join(data_dir, "2trading_etf_returns_aligned.csv")

factors = ["iwf", "mtum", "qual", "size", "usmv", "vlue"]   # used everywhere

grid_df    = pd.read_parquet("cv_params_grid.parquet")
bayes_df   = pd.read_parquet("cv_params_bayes_v2.parquet") # v2 is the one searching between 20-2000
history_df = pd.read_parquet("cv_params_history.parquet")

In [12]:
# ──────────────────────────────────────────────────────────────
# HYPERPARAMETERS
# ──────────────────────────────────────────────────────────────
df_map = {
    "grid":    grid_df,
    "bayes":   bayes_df,
    "history": history_df
}
cv_df = df_map[cv_choice]

# ─────────────────────────────────────────────────────
# HYPERPARAMETER SMOOTHING SETUP
# ─────────────────────────────────────────────────────
# pick one
SMOOTH_METHOD = "none"   # options: "none", "rolling_median", "ewma"
SMOOTH_WINDOW = 3        # # of folds to include in the window
# ─────────────────────────────────────────────────────

# … right after cv_df = df_map[cv_choice] …
cv_df = cv_df.sort_values(["factor","date"])

if SMOOTH_METHOD == "none":
    # simply copy original λ & κ forward
    cv_df["sm_lambda"] = cv_df["best_lambda"]
    cv_df["sm_kappa"]  = cv_df["best_kappa"]

elif SMOOTH_METHOD == "rolling_median":
    # Centered rolling median
    cv_df["sm_lambda"] = (
        cv_df
        .groupby("factor")["best_lambda"]
        .transform(lambda x: x.rolling(SMOOTH_WINDOW, min_periods=1, center=True).median())
    )
    cv_df["sm_kappa"] = (
        cv_df
        .groupby("factor")["best_kappa"]
        .transform(lambda x: x.rolling(SMOOTH_WINDOW, min_periods=1, center=True).median())
    )

elif SMOOTH_METHOD == "ewma":
    # Exponential‐weight moving average
    cv_df["sm_lambda"] = (
        cv_df
        .groupby("factor")["best_lambda"]
        .transform(lambda x: x.ewm(span=SMOOTH_WINDOW, min_periods=1).mean())
    )
    cv_df["sm_kappa"] = (
        cv_df
        .groupby("factor")["best_kappa"]
        .transform(lambda x: x.ewm(span=SMOOTH_WINDOW, min_periods=1).mean())
    )

else:
    raise ValueError(f"Unknown SMOOTH_METHOD {SMOOTH_METHOD!r}")

# round κ back to integer
cv_df["sm_kappa"] = cv_df["sm_kappa"].round().astype(int)

# overwrite with the chosen values
cv_df["best_lambda"] = cv_df["sm_lambda"]
cv_df["best_kappa"]  = cv_df["sm_kappa"]

# (optional) drop helpers
cv_df.drop(columns=["sm_lambda","sm_kappa"], inplace=True)

# ─────────────────────────────────────────────────────
# Now build saved_hyperparams exactly as before
# ─────────────────────────────────────────────────────
saved_hyperparams = {}
for fac in factors:
    sub = cv_df[cv_df["factor"] == fac].sort_values("date")
    saved_hyperparams[fac] = [
        {
            "date":      row["date"],
            "new_lambda": row["best_lambda"],
            "new_kappa":  row["best_kappa"]
        }
        for _, row in sub.iterrows()
    ]


In [13]:
# ──────────────────────────────────────────────────────────────
# DATA‑LOADING BLOCK  (pulled from old notebook)
# ──────────────────────────────────────────────────────────────

# 1) Load full data for every factor + market ------------------------------------------------
factor_data_dict  = {}
factor_returns_ls = []

for fac in factors:
    print(f"Loading data for {fac}")
    data = MergedDataLoader(
        factor_file=factor_file,
        market_file=market_file,
        ver="v2",
        factor_col=fac
    ).load()

    common_idx = (data.X.index
                  .intersection(data.ret_ser.index)
                  .intersection(data.market_ser.index))

    X_full        = data.X.loc[common_idx]
    fac_ret_full  = data.ret_ser.loc[common_idx]
    mkt_ret_full  = data.market_ser.loc[common_idx]
    active_ret    = fac_ret_full - mkt_ret_full

    factor_data_dict[fac] = {
        "X"        : X_full,
        "fac_ret"  : fac_ret_full,
        "mkt_ret"  : mkt_ret_full,
        "active_ret": active_ret,
    }
    factor_returns_ls.append(fac_ret_full)

# save last loop’s mkt_ret_full as market series
all_market_ret = mkt_ret_full

# 2) Assemble master return dataframe (factors + Market + rf) -------------------------------
full_factors_df = pd.concat(factor_returns_ls, axis=1).dropna()
full_df = pd.concat([full_factors_df, all_market_ret], axis=1).dropna()
full_df.columns = factors + ["Market"]

# risk‑free
etf_df   = pd.read_csv(etf_file, index_col=0, parse_dates=True).dropna().sort_index()
rf_ser   = etf_df["rf"]
full_df  = pd.concat([full_df, rf_ser], axis=1).dropna()
full_df.columns = factors + ["Market", "rf"]

# 3) Define test index (everything from 2017‑01‑01 on) --------------------------------------
test_slice = full_df.loc[test_start:]
test_index = test_slice.index.sort_values()
# ──────────────────────────────────────────────────────────────


Loading data for iwf
Loading data for mtum
Loading data for qual
Loading data for size
Loading data for usmv
Loading data for vlue


In [14]:
# ------------------------------------------------------------
# 1  BUILD & CACHE FACTOR‑VIEWS  (run once, takes minutes)
# ------------------------------------------------------------
VIEWS_FILE = "grid_factor_views_2014.pkl" # "SAVEfactor_views.pkl" is the views for the outperforming sharpe run
FORCE_REBUILD = False 

def _fit_one_factor(fac, refit_date, test_dates_chunk,
                    factor_data_dict, hyperparams,
                    min_years, max_years, init_start):

    # ---------- helpers ----------
    def get_train_window(current_date, full_data):
        train_end  = current_date
        train_start= max(train_end - pd.DateOffset(years=max_years),
                         pd.to_datetime(init_start))
        if (train_end - train_start) < pd.Timedelta(days=365.25*min_years):
            train_start = train_end - pd.DateOffset(years=min_years)
        idx = full_data.index
        subset = idx[(idx >= train_start) & (idx <= train_end)]
        start_date, end_date = subset.min(), subset.max()
        return start_date, end_date 

    # ---------- data ----------
    fac_data = factor_data_dict[fac]
    X   = fac_data["X"]
    ret = fac_data["fac_ret"]
    act = fac_data["active_ret"]

    lam = hyperparams["new_lambda"]
    kp  = hyperparams["new_kappa"]
    train_start, train_end = get_train_window(refit_date, X)

    # ---------- preprocess ----------
    clipper = DataClipperStd(mul=3.0)
    scaler  = StandardScaler()
    X_train = scaler.fit_transform(clipper.fit_transform(
                 filter_date_range(X, train_start, train_end)))
    active_train = filter_date_range(act, train_start, train_end)

    # ---------- fit SJM ----------
    sjm = SparseJumpModel(n_components=2,
                          max_feats=int(kp**2),
                          jump_penalty=lam)
    
    train_idx = filter_date_range(X, train_start, train_end).index
    X_train_df = pd.DataFrame(X_train, index=train_idx, columns=X.columns)
    sjm.fit(X_train_df, ret_ser=active_train, sort_by="cumret")

    ret_train = filter_date_range(ret, train_start, train_end)

    # regime‑level abs returns
    train_states = sjm.predict(X_train_df)
    abs_ret = {}
    for st in range(2):
        st_idx = (train_states==st)
        abs_ret[st] = ret_train.loc[st_idx].mean() * 252

    # ---------- online prediction for test dates ----------
    states = {}
    for day in test_dates_chunk:
        X_hist = X.loc[:day]                          # all history up to 'day'
        temp_clipper = DataClipperStd(mul=3.0)
        X_hist_clip  = temp_clipper.fit_transform(X_hist)

        temp_scaler  = StandardScaler()
        _ = temp_scaler.fit_transform(X_hist_clip)    # fit on *all* history

        if day in X.index:
            X_day_clip   = temp_clipper.transform(X.loc[[day]])
            X_day_scaled = temp_scaler.transform(X_day_clip)
            states[day]  = sjm.predict_online(
                pd.DataFrame(X_day_scaled,
                            index=[day],
                            columns=X.columns)).iloc[0]

    # assemble mini‑df for this factor & period
    out = pd.DataFrame({"state": pd.Series(states)},
                       index=list(states.keys()))
    out["ann_abs_ret"] = out["state"].map(abs_ret)
    return fac, out

def build_factor_views(factor_data_dict, saved_hyperparams, factors,
                       test_index,
                       refit_freq="ME", min_years=8, max_years=12,
                       init_start="2002-05-31"):

    views = {f:[] for f in factors}
    refit_dates = (test_index.to_series()
                   .resample(refit_freq)
                   .last()
                   .dropna())

    for j, refit_date in enumerate(refit_dates):
        if j < len(refit_dates)-1:
            next_refit = refit_dates.iloc[j+1]
        else:
            next_refit = test_index[-1]
        test_mask = (test_index>refit_date)&(test_index<=next_refit)
        test_chunk = test_index[test_mask]

        # ---- parallel over factors ----
        jobs = []
        for fac in factors:
            # latest hyperparams before refit_date
            hp_hist = [h for h in saved_hyperparams[fac]
                       if pd.to_datetime(h["date"])<=refit_date]
            if not hp_hist: continue
            hp = hp_hist[-1]
            jobs.append(delayed(_fit_one_factor)(
                fac, refit_date, test_chunk,
                factor_data_dict, hp,
                min_years, max_years, init_start))
        for fac, df in Parallel(n_jobs=-1)(jobs):
            views[fac].append(df)

    # concat & tidy
    for fac in factors:
        views[fac] = (pd.concat(views[fac])
                      .sort_index()
                      .loc[:,["state","ann_abs_ret"]])
    return views


# --------- build or load ----------         
if FORCE_REBUILD or not os.path.exists(VIEWS_FILE):
    factor_views = build_factor_views(factor_data_dict, saved_hyperparams, factors, 
                                      test_index,
                                      refit_freq=REFIT_FREQ, 
                                      min_years=8, max_years=12, init_start="2002-05-31")
    with open(VIEWS_FILE, "wb") as f:
        pickle.dump(factor_views, f)
else:
    with open(VIEWS_FILE, "rb") as f:
        factor_views = pickle.load(f)



In [15]:
# ------------------------------------------------------------
# 2  FAST BLACK‑LITTERMAN FUNCTION  (run as often as you like)
# ------------------------------------------------------------

def ewm_covariance(returns, halflife=126, min_periods=60):
    ewm_cov = returns.ewm(halflife=halflife,
                          adjust=False,
                          min_periods=min_periods).cov()
    if returns.empty: return pd.DataFrame()
    return ewm_cov.loc[returns.index[-1]]

def detect_state_shifts(views, factors):
    # 1 col per factor with the model‑state
    state_df = pd.concat({f: views[f]["state"] for f in factors}, axis=1)
    # True when *any* factor changes state vs. the day before
    return state_df.ne(state_df.shift()).any(axis=1)

def bl_max_sharpe_te(cov_hist, pi, views, tau, delta,
                     w_bmk, te_target, bounds, rf,
                     use_bl_cov=False):

    bl = BlackLittermanModel(cov_hist, pi=pi, tau=tau,
                             delta=delta, absolute_views=views)

    # pick the risk matrix
    Sigma = bl.bl_cov().values if use_bl_cov else cov_hist.values
    mu    = bl.bl_returns().values
    n     = len(mu)

    w = cp.Variable(n)
    w_act = w - w_bmk.values

    prob = cp.Problem(
        cp.Maximize((mu - rf) @ w),
        [
            cp.sum(w) == 1,
            w >= np.array([lo for lo, hi in bounds]),
            w <= np.array([hi for lo, hi in bounds]),
            cp.quad_form(w_act, Sigma) <= (te_target/np.sqrt(252))**2
        ]
    )
    prob.solve(solver="SCS")
    return pd.Series(w.value, index=w_bmk.index)


def run_bl_once(views, returns_df, full_df,
                shift_series=None,
                tau=0.05, delta=2.5,
                te_target=0.05, # 5 % TE
                trade_market=True,
                use_bl_cov=False,
                allow_market_short=False,
                allow_factor_short=False,
                use_bl_prior=False,     
                fallback_strategy="HOLD_RFR", # "HOLD_RFR", "SHORT_MARKET"
                tcost=0.0005):

    assets  = returns_df.columns.tolist()
    factors = list(views.keys())
    if trade_market:
        trade_assets = [a for a in returns_df.columns if a != "rf"]
    else:
        trade_assets = [a for a in returns_df.columns
                        if a not in {"rf", "Market"}]
    cash_asset = "rf"

    market_caps = {etf: 1.0 for etf in trade_assets}

    # ---------- per‑asset bounds ----------
    bounds = []
    for a in trade_assets:
        if a == "Market":
            bounds.append((-1, 1) if allow_market_short else (0, 1))
        else:
            bounds.append((-1, 1) if allow_factor_short else (0, 1))

    w = pd.DataFrame(index=returns_df.index,
                     columns=trade_assets + [cash_asset], # was + [cash_asset]
                     dtype=float)

    for t in returns_df.index:
        # ------ carry weights forward if no trade today ------
        if shift_series is not None and not shift_series.loc[t]:
                w.loc[t] = w.shift(1).loc[t] 
                continue
        
        # ------ optimiser block ------
        hist  = full_df[trade_assets].loc[:t].iloc[:-1]
        cov   = ewm_covariance(hist) * 252
        if cov.empty or cov.isna().any().any():
            continue

        if use_bl_prior:
            prior_for_bl = market_implied_prior_returns(market_caps, delta, cov)
        else:
            prior_for_bl = "equal"          # or None


        q = {fac: views[fac].loc[t, "ann_abs_ret"] for fac in factors}

        
        # ----- ***lightweight BL*** just for the fallback test ------------
        bl0 = BlackLittermanModel(cov, pi=prior_for_bl,
                                tau=tau, delta=delta,
                                absolute_views=q)

        rf_annual = returns_df.loc[t, cash_asset] * 252

        # ===== fallback: all ERs ≤ cash ===================================
        if (bl0.bl_returns() <= rf_annual).all():
            w.loc[t] = 0.0
            if fallback_strategy == "HOLD_RFR":
                w.loc[t, cash_asset] = 1.0
            elif (fallback_strategy == "SHORT_MARKET"
                and "Market" in trade_assets):
                w.loc[t, "Market"] = -1.0
                w.loc[t, cash_asset] = 1.0
            continue               # ← next date
        # =================================================================

        # ---- target-TE BL optimisation --------------------------------
        w_bmk = pd.Series(1/len(trade_assets), index=trade_assets)

        w_t = bl_max_sharpe_te(cov,
                            pi=prior_for_bl,
                            views=q,
                            tau=tau, delta=delta,
                            w_bmk=w_bmk,
                            te_target=te_target,          # 5 % TE
                            bounds=bounds,
                            rf=rf_annual,
                            use_bl_cov=use_bl_cov)
        w.loc[t, trade_assets] = w_t

    # ---------- P&L ----------
    pnl = (w.shift(1).fillna(0) * returns_df).sum(axis=1)
    if tcost > 0:
        pnl -= w.diff().abs().sum(axis=1).fillna(0) * tcost

    return w, pnl


In [16]:
# ------------------------------------------------------------
# 3  QUICK EXPERIMENTS
# ------------------------------------------------------------
def annualized_sharpe(r):          # helper
    return (r.mean() / r.std()) * np.sqrt(252)

def ann_turnover(w):
    daily_turn = w.diff().abs().sum(axis=1).mean()
    return daily_turn * 252

test_df = full_df.loc[test_index]
shift_days = detect_state_shifts(factor_views, factors).reindex(test_df.index, fill_value=False)
shift_days.iloc[0] = True

cfgs = [
    # these are the standard BL cov and BL prior that got 94 SR - now just trying different TE
     dict(label="94 SR + BL cov + BL prior - 5% TE",  tau=0.05, delta=2.5,
         use_bl_cov=True, use_bl_prior=True, allow_market_short=False, allow_factor_short=False, te_target=0.05),
     dict(label="94 SR + BL cov + BL prior - 4% TE",  tau=0.05, delta=2.5,
         use_bl_cov=True, use_bl_prior=True, allow_market_short=False, allow_factor_short=False, te_target=0.04),
     dict(label="94 SR + BL cov + BL prior - 3% TE",  tau=0.05, delta=2.5,
         use_bl_cov=True, use_bl_prior=True, allow_market_short=False, allow_factor_short=False, te_target=0.03),
     dict(label="94 SR + BL cov + BL prior - 2% TE",  tau=0.05, delta=2.5,
         use_bl_cov=True, use_bl_prior=True, allow_market_short=False, allow_factor_short=False, te_target=0.02),     
     dict(label="94 SR + BL cov + BL prior - 1% TE",  tau=0.05, delta=2.5,
         use_bl_cov=True, use_bl_prior=True, allow_market_short=False, allow_factor_short=False, te_target=0.01),

     

     # standard BL cov and BL prior that got 94 SR - now allowing shorts
     dict(label="94 SR + BL cov + BL prior - L/S mkt and fac",  tau=0.05, delta=2.5,
         use_bl_cov=True, use_bl_prior=True, allow_market_short=True, allow_factor_short=True, te_target=0.03),
     dict(label="94 SR + BL cov + BL prior - L/S mkt L fac",  tau=0.05, delta=2.5,
         use_bl_cov=True, use_bl_prior=True, allow_market_short=True, allow_factor_short=False, te_target=0.03),

     # standard BL cov and BL prior that got 94 SR - now only trading on regime shifts
     dict(label="94 SR + BL cov + BL prior - only trade on shift",  tau=0.05, delta=2.5, shift_series=shift_days,
         use_bl_cov=True, use_bl_prior=True, allow_market_short=False, allow_factor_short=False, te_target=0.03),

     # standard set ups from 94SR run - now falling back on market short
     dict(label="94 SR + BL cov + BL prior - fallback mkt short",  tau=0.05, delta=2.5, shift_series=shift_days,
         use_bl_cov=True, use_bl_prior=True, allow_market_short=True, allow_factor_short=False, te_target=0.03,
         fallback_strategy="SHORT_MARKET"),
     dict(label="Long only base - 3% TE now with daily trade",  tau=0.05, delta=2.5,shift_series=shift_days,
         use_bl_cov=False, use_bl_prior=False, allow_market_short=False, allow_factor_short=False, te_target=0.03),
     
]


run_results = {}                   # label → dict(rets, wts, cfg)
for c in cfgs:
    label = c.pop("label")         # remove label before **c
    wts, rets = run_bl_once(factor_views, test_df, full_df, **c)  # set shift_series to None to trade daily. set to shift_days to only trade on regime shifts
    run_results[label] = dict(returns=rets, weights=wts, cfg=c)
    #print(f"{label:12s}  Sharpe {annualized_sharpe(rets):6.3f}")

print()

rows = []
for label, res in run_results.items():
    rows.append({
        "Strategy": label,
        "Sharpe": annualized_sharpe(res["returns"]),
        "Turnover": ann_turnover(res["weights"])
    })

# make and print the table
df_table   = pd.DataFrame(rows)
print(df_table.to_string(index=False,float_format=lambda x: f"{x:.3f}"))





/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/probl


                                       Strategy  Sharpe  Turnover
              94 SR + BL cov + BL prior - 5% TE   0.659     9.172
              94 SR + BL cov + BL prior - 4% TE   0.662     8.386
              94 SR + BL cov + BL prior - 3% TE   0.666     7.602
              94 SR + BL cov + BL prior - 2% TE   0.669     6.826
              94 SR + BL cov + BL prior - 1% TE   0.673     6.057
    94 SR + BL cov + BL prior - L/S mkt and fac   0.666     7.604
      94 SR + BL cov + BL prior - L/S mkt L fac   0.666     7.603
94 SR + BL cov + BL prior - only trade on shift   0.661     7.153
 94 SR + BL cov + BL prior - fallback mkt short   0.250    12.208
    Long only base - 3% TE now with daily trade   0.672     6.578


In [17]:
# These results are from 5% TE with the historical crossvals

# build a list of dicts from your run_results
rows = []
for label, res in run_results.items():
    rows.append({
        "Strategy": label,
        "Sharpe": annualized_sharpe(res["returns"]),
        "Turnover": ann_turnover(res["weights"])
    })

# make and print the table
df_table   = pd.DataFrame(rows)
print(df_table.to_string(index=False,float_format=lambda x: f"{x:.3f}"))

                                       Strategy  Sharpe  Turnover
              94 SR + BL cov + BL prior - 5% TE   0.659     9.172
              94 SR + BL cov + BL prior - 4% TE   0.662     8.386
              94 SR + BL cov + BL prior - 3% TE   0.666     7.602
              94 SR + BL cov + BL prior - 2% TE   0.669     6.826
              94 SR + BL cov + BL prior - 1% TE   0.673     6.057
    94 SR + BL cov + BL prior - L/S mkt and fac   0.666     7.604
      94 SR + BL cov + BL prior - L/S mkt L fac   0.666     7.603
94 SR + BL cov + BL prior - only trade on shift   0.661     7.153
 94 SR + BL cov + BL prior - fallback mkt short   0.250    12.208
    Long only base - 3% TE now with daily trade   0.672     6.578


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import SelectMultiple, Dropdown, ToggleButtons, VBox, HBox, interact

# 0) remove any old EW entries
for k in list(run_results):
    if k.startswith("Quarterly EW"):
        run_results.pop(k)

# 1) compute quarterly EW over ONLY the risky assets (no rf)
TCOST = 0.0005
df    = etf_df.loc[test_index.intersection(etf_df.index)].drop(columns="rf")
n     = df.shape[1]

# quarter-end dates
first = df.index[0]
qends = (df.index.to_series()
           .resample("QE").last()
           .dropna()
           .index)
if first not in qends:
    qends = qends.insert(0, first)

ew_rets    = pd.Series(index=df.index, dtype=float)
ew_weights = pd.DataFrame(index=df.index, columns=df.columns, dtype=float)
prev_w     = np.zeros(n)

for i in range(len(qends)-1):
    start, end = qends[i], qends[i+1]
    period      = df.loc[(df.index>start)&(df.index<=end)]
    w           = np.ones(n)/n
    for j, day in enumerate(period.index):
        ew_weights.loc[day] = w
        r_i  = period.loc[day].values
        tc   = TCOST * np.abs(w-prev_w).sum() if j==0 else 0.0
        p    = w.dot(r_i) - tc
        ew_rets.loc[day] = p
        w    = w*(1+r_i)
        if (1+p)!=0:
            w /= (1+p)
    prev_w = w.copy()

ew_rets = ew_rets.dropna()
ew_cum  = ew_rets.cumsum()

# 2) build active‐dates mask and trim weights
active = pd.Index([])
for cfg in run_results.values():
    idx = cfg["weights"].drop(columns="rf", errors="ignore") \
                        .dropna(how="all").index
    active = active.union(idx)
active = active.sort_values()

ew_w = ew_weights.loc[ew_weights.index.isin(active)]

# inject the single EW portfolio
run_results["Quarterly EW (no rf, 5bps)"] = {
    "returns": ew_rets,
    "weights": ew_w
}

# 3) widgets
labels = list(run_results)
cmp    = SelectMultiple(options=labels,
                       value=tuple(labels[:2]),
                       description="Compare:",
                       rows=min(8,len(labels)),
                       style={"description_width":"70px"})
wgt    = Dropdown(options=labels,
                  value=labels[0],
                  description="Weights:",
                  style={"description_width":"70px"})
sign   = ToggleButtons(options=[("Both","both"),("Positive","pos"),("Negative","neg")],
                       value="both",
                       description="Show:",
                       style={"description_width":"70px"})

def sharpe(x):
    return x.mean()/x.std()*np.sqrt(252)

def _update(compare, weights_cfg, sign_filter):
    if not compare:
        print("Pick ≥1 config."); return

    # cumulative returns
    plt.figure(figsize=(12,6))
    ew_cum.plot(label="Quarterly EW (no rf, 5bps)", lw=2)
    for lab in compare:
        run_results[lab]["returns"].cumsum().plot(label=lab, lw=1.5)
    plt.title("Cumulative Returns"); plt.grid(); plt.legend(); plt.show()

    # Sharpe table
    print("Annualised Sharpe ratios")
    print(f"  EW            : {sharpe(ew_rets):.3f}")
    for lab in compare:
        print(f"  {lab:<12s}: {sharpe(run_results[lab]['returns']):.3f}")

    # weights over time
    wdf = (run_results[weights_cfg]["weights"]
           .dropna(how="all"))
    if sign_filter=="pos":
        wdf = wdf.where(wdf>0,0)
    elif sign_filter=="neg":
        wdf = wdf.where(wdf<0,0)

    plt.figure(figsize=(12,6))
    plt.stackplot(wdf.index, wdf.T.values, labels=wdf.columns)
    filt = {"both":"(all)","pos":"(positive)","neg":"(negative)"}[sign_filter]
    plt.title(f"Weights over time – {weights_cfg} {filt}")
    plt.xlabel("Trading days"); plt.ylabel("Weight")
    plt.legend(loc="center left", bbox_to_anchor=(1,.5), fontsize="small")
    plt.tight_layout(); plt.show()

ui = VBox([HBox([cmp, wgt, sign])])
interact(_update, compare=cmp, weights_cfg=wgt, sign_filter=sign)

# 4) realized TE + summary
def te(pnl, bench):
    a,b = pnl.align(bench, join="inner")
    return np.sqrt(252)*(a-b).std()

rows = []
for lab,res in run_results.items():
    rows.append({
        "Strategy"   : lab,
        "Sharpe"     : sharpe(res["returns"]),
        "Turnover"   : ann_turnover(res["weights"]),
        "Realized TE": te(res["returns"], ew_rets)
    })

df = pd.DataFrame(rows)
print(df.to_string(index=False, float_format=lambda x: f"{x:.3f}"))




interactive(children=(SelectMultiple(description='Compare:', index=(0, 1), options=('94 SR + BL cov + BL prior…

                                       Strategy  Sharpe  Turnover  Realized TE
              94 SR + BL cov + BL prior - 5% TE   0.659     9.172        0.120
              94 SR + BL cov + BL prior - 4% TE   0.662     8.386        0.120
              94 SR + BL cov + BL prior - 3% TE   0.666     7.602        0.120
              94 SR + BL cov + BL prior - 2% TE   0.669     6.826        0.120
              94 SR + BL cov + BL prior - 1% TE   0.673     6.057        0.120
    94 SR + BL cov + BL prior - L/S mkt and fac   0.666     7.604        0.120
      94 SR + BL cov + BL prior - L/S mkt L fac   0.666     7.603        0.120
94 SR + BL cov + BL prior - only trade on shift   0.661     7.153        0.119
 94 SR + BL cov + BL prior - fallback mkt short   0.250    12.208        0.238
    Long only base - 3% TE now with daily trade   0.672     6.578        0.115
                     Quarterly EW (no rf, 5bps)   0.618     0.657        0.000


In [21]:
# 5) Sharpe comparison tests vs EW
def sharpe_z_test(pnl, bench, periods=252):
    idx = pnl.index.intersection(bench.index)
    x, y = pnl.loc[idx], bench.loc[idx]
    n    = len(x)
    sr_p = x.mean()/x.std(ddof=1)*np.sqrt(periods)
    sr_b = y.mean()/y.std(ddof=1)*np.sqrt(periods)
    var_p = (1 + 0.5*sr_p**2)/n
    var_b = (1 + 0.5*sr_b**2)/n
    z     = (sr_p - sr_b)/np.sqrt(var_p+var_b)
    p     = 2*(1 - stats.norm.cdf(abs(z)))
    return sr_p, sr_b, z, p

def sharpe_bootstrap_test(pnl, bench, periods=252, n_boot=10000, seed=0):
    idx = pnl.index.intersection(bench.index)
    x, y = pnl.loc[idx].to_numpy(), bench.loc[idx].to_numpy()
    n    = len(x)
    sr1  = x.mean()/x.std(ddof=1)*np.sqrt(periods)
    sr2  = y.mean()/y.std(ddof=1)*np.sqrt(periods)
    obs  = sr1 - sr2
    rng  = np.random.default_rng(seed)
    diffs = np.empty(n_boot)
    for i in range(n_boot):
        sel = rng.integers(0,n,size=n)
        x_b, y_b = x[sel], y[sel]
        sr1b = x_b.mean()/x_b.std(ddof=1)*np.sqrt(periods)
        sr2b = y_b.mean()/y_b.std(ddof=1)*np.sqrt(periods)
        diffs[i] = sr1b - sr2b
    p = 2*np.mean(diffs<=0) if obs>0 else 2*np.mean(diffs>=0)
    return obs, p

bench = ew_rets
print("\nParametric Z-test vs EW:")
for label, res in run_results.items():
    sr_p, sr_b, z, p = sharpe_z_test(res["returns"], bench)
    print(f"{label:25s} SR={sr_p:.3f} vs EW={sr_b:.3f}, z={z:+.2f}, p={p:.3f}")

print("\nBootstrap test vs EW:")
for label, res in run_results.items():
    diff, p = sharpe_bootstrap_test(res["returns"], bench)
    print(f"{label:25s} ΔSR={diff:+.3f}, p={p:.3f}")



Parametric Z-test vs EW:
94 SR + BL cov + BL prior - 5% TE SR=0.659 vs EW=0.618, z=+1.39, p=0.165
94 SR + BL cov + BL prior - 4% TE SR=0.662 vs EW=0.618, z=+1.51, p=0.132
94 SR + BL cov + BL prior - 3% TE SR=0.666 vs EW=0.618, z=+1.63, p=0.104
94 SR + BL cov + BL prior - 2% TE SR=0.669 vs EW=0.618, z=+1.74, p=0.081
94 SR + BL cov + BL prior - 1% TE SR=0.673 vs EW=0.618, z=+1.86, p=0.063
94 SR + BL cov + BL prior - L/S mkt and fac SR=0.666 vs EW=0.618, z=+1.63, p=0.104
94 SR + BL cov + BL prior - L/S mkt L fac SR=0.666 vs EW=0.618, z=+1.63, p=0.104
94 SR + BL cov + BL prior - only trade on shift SR=0.661 vs EW=0.618, z=+1.47, p=0.143
94 SR + BL cov + BL prior - fallback mkt short SR=0.250 vs EW=0.618, z=-13.04, p=0.000
Long only base - 3% TE now with daily trade SR=0.672 vs EW=0.618, z=+1.84, p=0.066
Quarterly EW (no rf, 5bps) SR=0.618 vs EW=0.618, z=+0.00, p=1.000

Bootstrap test vs EW:
94 SR + BL cov + BL prior - 5% TE ΔSR=+0.041, p=0.870
94 SR + BL cov + BL prior - 4% TE ΔSR=+0.044,

In [ ]:
from scipy.stats import wilcoxon

# 1) align your strategy vs. EW returns
def align_returns(pnl, bench):
    idx = pnl.index.intersection(bench.index)
    return pnl.loc[idx], bench.loc[idx]

# 2) Wilcoxon signed-rank test
for label, res in run_results.items():
    strat, ew = align_returns(res["returns"], quarterly_ew_rets)
    diff = strat - ew
    stat, p = wilcoxon(diff)
    print(f"{label:25s} Wilcoxon stat={stat:.3f}, p-value={p:.3f}")

Long only base - 5% TE    Wilcoxon stat=1050165.000, p-value=0.820
Long only base - 4% TE    Wilcoxon stat=1047997.000, p-value=0.758
Long only base - 3% TE    Wilcoxon stat=1045877.000, p-value=0.699
Long only base - 2% TE    Wilcoxon stat=1043829.000, p-value=0.644
Long only base - 1% TE    Wilcoxon stat=1041709.000, p-value=0.588
+ BL prior                Wilcoxon stat=1038405.000, p-value=0.507
+ BL cov                  Wilcoxon stat=1049915.000, p-value=0.813
+ BL cov + BL prior       Wilcoxon stat=1038218.000, p-value=0.502
Quarterly EW (no rf, 5bps) Wilcoxon stat=957578.000, p-value=0.000
